In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "xy_r_theta/"
create_directory(data_directory)
create_directory("models/xy_r_theta/")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 60
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = 16    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
all_trials = read_dataset("data/r_passive_trials.h5", cols=BASIC_TRAINING_COLS)
np.random.seed(0)
idx = np.random.permutation(np.arange(len(all_trials)))
train_idx = idx[:3500]
val_idx = idx[3500:4500]

train_trials = []

for index in train_idx:
    train_trials.append(all_trials[index])
    
val_trials = []

for index in val_idx:
    val_trials.append(all_trials[index])

100%|██████████| 10935/10935 [01:21<00:00, 133.58it/s]


In [6]:
XY_RTHETA_ATTRS = [obj+"."+attr for obj in ["o1", "o2", "o3", "o4"] for attr in ["x", "y", "r", "theta"]]

def add_r_theta_attributes(trials):
    for trial in tqdm(trials):
        for obj in ["o1", "o2", "o3", "o4"]:
            trial[obj+".r"] = (trial[obj+".vx"]**2 + trial[obj+".vy"]**2)**0.5
            trial[obj+".theta"] = (np.arctan2(trial[obj+".vx"], trial[obj+".vy"]) * 180 / np.pi)

add_r_theta_attributes(train_trials)
add_r_theta_attributes(val_trials)

100%|██████████| 1000/1000 [01:37<00:00,  9.79it/s]


## Train model and plot loss and accuracy

In [7]:
N_MODELS = 25

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device, training_columns=XY_RTHETA_ATTRS)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/xy_r_theta/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/xy_r_theta/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:02<00:00, 453.46it/s]
Train_loss: ([0.43314376 0.29800404]) Train_acc: ([78.42857143 82.05714286]) Val_acc: ([67.7 70.4]): 100%|██████████| 60/60 [09:31<00:00,  9.49s/it]
Train_loss: ([0.48517282 0.24545256]) Train_acc: ([75.42857143 86.57142857]) Val_acc: ([66.5 71.1]): 100%|██████████| 60/60 [09:37<00:00,  9.68s/it]
Train_loss: ([0.75660718 0.60562564]) Train_acc: ([63.28571429 68.34285714]) Val_acc: ([63.1 65.8]): 100%|██████████| 60/60 [09:41<00:00,  9.52s/it]
Train_loss: ([0.54875568 0.41717471]) Train_acc: ([73.85714286 80.42857143]) Val_acc: ([66.8 73.6]): 100%|██████████| 60/60 [09:36<00:00,  9.72s/it]
Train_loss: ([0.5658053  0.44911008]) Train_acc: ([71.68571429 76.31428571]) Val_acc: ([65.3 71.1]): 100%|██████████| 60/60 [09:33<00:00,  9.57s/it]
Train_loss: ([0.50242798 0.37826234]) Train_acc: ([76.85714286 73.28571429]) Val_acc: ([68.5 66.5]): 100%|██████████| 60/60 [09:41<00:00,  9.81s/it]
Train_loss: ([0.46502943 0.29255699]) Train_acc: ([79

## Save model and scaler

In [8]:
joblib.dump(scaler, "scalers/passive_xy_rtheta_scaler.sk")

['scalers/passive_xy_rtheta_scaler.sk']